# Análise Exploratória — Experimento A/B de Campanha Digital

## Contexto de Negócio

Uma empresa realizou um experimento A/B com o objetivo de avaliar o impacto de um anúncio digital na taxa de conversão dos usuários.

- Grupo **ad**: usuários expostos ao anúncio.
- Grupo **psa**: usuários não expostos (grupo controle).

O objetivo desta etapa é realizar uma análise exploratória dos dados (EDA) para:

- Compreender a estrutura do dataset
- Avaliar qualidade e consistência dos dados
- Explorar diferenças iniciais entre os grupos
- Identificar possíveis vieses ou desequilíbrios

A validação estatística formal da diferença será realizada no próximo notebook.

## Objetivos Analíticos

Nesta análise buscamos responder:

1. Os grupos possuem tamanhos balanceados?
2. Existem diferenças visíveis na taxa de conversão?
3. Há indícios de viés na distribuição de variáveis demográficas?
4. A diferença observada parece relevante antes do teste estatístico?

## 1. Importação das Bibliotecas

In [1]:
import pandas as pd
import matplotlib.pyplot as plt

## 2. Carregamento dos Dados

In [2]:
df = pd.read_csv('../data/marketing_AB.csv')

In [33]:
df.sample(10)

,Unnamed: 0,user id,test group,converted,total ads,most ads day,most ads hour
300861,300861,1346090,ad,False,2,Friday,21
409762,409762,1412897,ad,False,1,Wednesday,15
421209,421209,1619558,ad,False,1,Thursday,18
381825,381825,1429354,ad,False,13,Wednesday,19
155904,155904,1526530,ad,False,51,Friday,13
250969,250969,1059940,ad,False,6,Saturday,12
244468,244468,1209344,ad,False,31,Sunday,11
229076,229076,1446404,ad,False,5,Friday,18
92272,92272,1286778,ad,False,38,Monday,14
17258,17258,1256947,ad,False,243,Friday,15


## 3. Visão Geral do Dataset

In [4]:
df.shape

(588101, 7)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 588101 entries, 0 to 588100
Data columns (total 7 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   Unnamed: 0     588101 non-null  int64 
 1   user id        588101 non-null  int64 
 2   test group     588101 non-null  object
 3   converted      588101 non-null  bool  
 4   total ads      588101 non-null  int64 
 5   most ads day   588101 non-null  object
 6   most ads hour  588101 non-null  int64 
dtypes: bool(1), int64(4), object(2)
memory usage: 27.5+ MB


In [6]:
df.isnull().sum()

Unnamed: 0       0
user id          0
test group       0
converted        0
total ads        0
most ads day     0
most ads hour    0
dtype: int64

## 4. Balanceamento do Experimento

Antes de analisar conversão, precisamos garantir que os grupos são comparáveis.

In [7]:
df['test group'].value_counts()

test group
ad     564577
psa     23524
Name: count, dtype: int64

In [8]:
df['test group'].value_counts(normalize=True)

test group
ad     0.96
psa    0.04
Name: proportion, dtype: float64

## 5. Métrica Principal: Conversão

In [9]:
df['converted'].mean()

np.float64(0.02523886203220195)

 ### 5.2 Taxa de Conversão por Grupo

In [11]:
summary = df.groupby('test group')['converted'].agg(['count', 'sum','mean',])
summary.rename(columns={'mean': 'conversion_rate'}, inplace=True)
summary

,count,sum,conversion_rate
test group,,,
ad,564577,14423,0.025547
psa,23524,420,0.017854


### 5.3 Diferença Absoluta e Relativa

In [12]:
p_ad = summary.loc['ad', 'conversion_rate']
p_psa = summary.loc['psa', 'conversion_rate']

absolute_diff = p_ad - p_psa
relative_diff = absolute_diff / p_psa

absolute_diff, relative_diff

(np.float64(0.007692453192201517), np.float64(0.43085064022225833))

## 6. Validação de Randomização e Possível Viés

### 6.1 Distribuição de Dias por Grupo

In [13]:
pd.crosstab(df['most ads day'], df['test group'], normalize='columns')

test group,ad,psa
most ads day,,
Friday,0.157295,0.161665
Monday,0.148024,0.148869
Saturday,0.139577,0.121493
Sunday,0.145830,0.130037
Thursday,0.140064,0.166001
Tuesday,0.132085,0.123576
Wednesday,0.137126,0.148359


### 6.2 Distribuição de Horários por Grupo

In [14]:
pd.crosstab(df['most ads hour'], df['test group'], normalize='columns')

test group,ad,psa
most ads hour,,
0,0.009404,0.009650
1,0.008174,0.007949
2,0.009125,0.007694
3,0.004588,0.003783
4,0.001229,0.001190
5,0.001314,0.000978
6,0.003516,0.003528
7,0.010925,0.010075
8,0.030054,0.028014


### 6.3 Comparação de Exposição (Total de Ads)

In [ ]:
df.groupby('test group')['total ads'].describe()

,count,mean,std,min,25%,50%,75%,max
test group,,,,,,,,
ad,564577.0,24.823365,43.750456,1.0,4.0,13.0,27.0,2065.0
psa,23524.0,24.761138,42.860720,1.0,4.0,12.0,26.0,907.0


## 7. Verificação das Condições para Teste de Proporções

In [16]:
summary['n*p'] = summary['count'] * summary['conversion_rate']
summary['n*(1-p)'] = summary['count'] * (1 - summary['conversion_rate'])

summary

,count,sum,conversion_rate,n*p,n*(1-p)
test group,,,,,
ad,564577,14423,0.025547,14423.0,550154.0
psa,23524,420,0.017854,420.0,23104.0


## Conclusões da Análise Exploratória

O experimento apresenta forte desbalanceamento entre os grupos (96% tratamento vs 4% controle). 
Apesar disso, o tamanho absoluto do grupo controle (23 mil observações) é suficientemente grande para permitir inferência estatística robusta.

A taxa de conversão observada foi:

- Grupo exposto ao anúncio (ad): 2,55%
- Grupo controle (psa): 1,79%

A diferença absoluta é de aproximadamente 0,77 ponto percentual, representando um aumento relativo de 43% na conversão.

As análises de distribuição por dia e horário indicam padrões semelhantes entre os grupos, sugerindo randomização adequada. 
Além disso, a média de exposição a anúncios é praticamente idêntica entre os grupos, reduzindo risco de viés por intensidade de tratamento.

As condições para aplicação do teste de proporções (n*p e n*(1-p) > 5) são plenamente atendidas.

Portanto, os dados estão adequados para prosseguir com a etapa de teste estatístico formal, a fim de avaliar se a diferença observada é estatisticamente significativa.